In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os

Mounted at /content/drive


In [2]:
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import numpy as np

In [3]:
import os
import torch
import torchvision
from torchvision import transforms
from PIL import Image
import numpy as np

# Load a pre-trained object detection model
model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)
model.eval()

# Transformation for input images
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Directory containing images
image_dir = '/content/drive/My Drive/Research/Train Masks/'
output_dir = '/content/drive/My Drive/Research/Train Label/'
os.makedirs(output_dir, exist_ok=True)

# Confidence threshold
confidence_threshold = 0.5

# Process each image in the dataset directory
for filename in os.listdir(image_dir):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_dir, filename)
        image = Image.open(image_path).convert("RGB")
        image_tensor = transform(image)

        # Run detection
        with torch.no_grad():
            outputs = model([image_tensor])
        outputs = outputs[0]
        boxes = outputs['boxes']
        scores = outputs['scores']
        labels = outputs['labels']

        np_image = np.array(image)

        for idx, score in enumerate(scores):
            if score >= confidence_threshold:
                label = labels[idx].item()
                box = boxes[idx].cpu().numpy().astype(int)

                xmin, ymin, xmax, ymax = box
                # Ensure coordinates are within image bounds
                xmin = max(0, xmin)
                ymin = max(0, ymin)
                xmax = min(np_image.shape[1], xmax)
                ymax = min(np_image.shape[0], ymax)

                entity_image = np_image[ymin:ymax, xmin:xmax]
                entity_pil = Image.fromarray(entity_image)

                # Save with a filename indicating source image and detection index
                save_path = os.path.join(
                    output_dir,
                    f"{os.path.splitext(filename)[0]}_entity_{idx}_score_{score:.2f}.png"
                )
                entity_pil.save(save_path)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth
100%|██████████| 170M/170M [00:01<00:00, 125MB/s]
